# Using HealthOmics Storage with genomic references and readsets
### This is the second notebook (2 of 3) in the workshop series and should be run AFTER the ECR setup notebook but BEFORE the workflow notebook. 
The goal of this notebook is to get you acquainted with HealthOmics Storage.

______________________________________________________
#### If you complete this notebook you will have:
+ Created a Reference Store
+ Imported a Reference Genome
+ Created a Sequence Store
+ Imported FASTQ files

## Prerequisites
#### Python requirements
+ Python >= 3.8
#### Packages:
+ boto3 >= 1.26.19
+ botocore >= 1.29.19
#### AWS requirements
+ AWS CLI
+ You will need the AWS CLI installed and configured in your environment. Supported AWS CLI versions are:
    - AWS CLI v2 >= 2.9.3 (Recommended)
    - AWS CLI v1 >= 1.27.19
    - AWS Region

<div class="alert alert-block alert-info">
<b>NOTE:</b> AWS HealthOmics only allows importing data within the same region. AWS HealthOmics is currently available in Oregon (us-west-2), N. Virginia (us-east-1), Dublin (eu-west-1), London (eu-west-2), Frankfurt (eu-central-1), and Singapore (ap-southeast-1).</div>

## Getting Started
### Step 1. Import libraries

In [ ]:
#Import necessary libraries and python SDK
from datetime import datetime
import json
import os
import time

import boto3
import botocore.exceptions

### Step 2. Setup new role
For the purposes of this demo, we will use the following policy and trust policy that are rather permissiv. You will need to customize permissions as required.

In [ ]:
# Define demo policies
storage_demo_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:*",
                "omics:*",
            ],
            "Resource": "*"
        }
    ]
}

storage_demo_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": ["sagemaker.amazonaws.com", "omics.amazonaws.com"]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

In [ ]:
# We will use this as the base name for our role and policy
omics_iam_name = 'OmicsStoreDemoRole'

# Create the iam client
iam = boto3.resource('iam')

# Check if the role already exists; if not, create it
try:
    role = iam.Role(omics_iam_name)
    role.load()
    
except botocore.exceptions.ClientError as ex:
    if ex.response["Error"]["Code"] == "NoSuchEntity":
        #Create the role with the corresponding trust policy
        role = iam.create_role(
            RoleName=omics_iam_name, 
            AssumeRolePolicyDocument=json.dumps(storage_demo_trust_policy))
        
        #Create policy
        policy = iam.create_policy(
            PolicyName='{}-policy'.format(omics_iam_name), 
            Description="Policy for AWS HealthOmics demo",
            PolicyDocument=json.dumps(storage_demo_policy))
        
        #Attach the policy to the role
        policy.attach_role(RoleName=omics_iam_name)
    else:
        print('Something went wrong, please retry and check your account settings and permissions')

In [ ]:
#Retrieve the role arn, which grants AWS HealthOmics the proper permissions to access the resources it needs in your AWS account.
def get_role_arn(role_name):
    try:
        iam = boto3.resource('iam')
        role = iam.Role(role_name)
        role.load()  # calls GetRole to load attributes
    except botocore.exceptions.ClientError:
        print("Couldn't get role named %s."%role_name)
        raise
    else:
        print(role.arn)
        return role.arn

In [ ]:
#Print role name and role arn to be used in store creation and upload
role_arn = get_role_arn(omics_iam_name)

In [ ]:
#Retrieve the region in which we are running our notebook.
region = boto3.session.Session().region_name
print(region)

### Step 3. Setup the HealthOmics client

In [ ]:
omics = boto3.client('omics', region_name=region)

### Step 4. Make demo data bucket and move scrnaseq test data to your bucket

In [ ]:
#e.g. s3://nigms-scrnaseq-bucket-demo-data
!aws s3 mb s3://[YOUR-BUCKET] #Replace

In [ ]:
!aws s3 cp s3://aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0/samplesheet-2-0.csv s3://[YOUR-BUCKET]
!aws s3 cp s3://aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0/GRCm38.p6.genome.chr19.fa s3://[YOUR-BUCKET]
!aws s3 cp s3://aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0/gencode.vM19.annotation.chr19.gtf s3://[YOUR-BUCKET]

--------------------------------------------
## Reference Store
Now we are going to setup a reference store. The reference genome we will be using for this demo is part of nf-core's methylseq test data hosted on their [Github](https://github.com/nf-core/test-datasets/tree/methylseq).

### Step 5. Create new reference store
Let's create a helper method to retrieve the reference store id and have it return empty if it doesn't exist. There should only be one reference store per region per account.<br>
For the purposes of this demo we will be providing inputs from S3, but the reference and sequence stores provide more efficient storage options.

<div class="alert alert-block alert-info">
<b>NOTE:</b> If this is the first time you've created a reference, you first must create a reference store using the code below. If a reference store already exists the code above will let you know.</div>

In [ ]:
def get_ref_store_id(client=None):
    if not client:
        client = boto3.client('omics')
    resp = client.list_reference_stores(maxResults=10)
    list_of_stores = resp.get('referenceStores')
    store_id = None
    if list_of_stores != None:
        # As mentioned above there can only be one store per region
        # if there is a store present, it is the first one
        store_id = list_of_stores[0].get('id')
    return store_id

In [ ]:
print(f"Checking for a reference store in region: {omics.meta.region_name}")
if get_ref_store_id(omics) == None:
    response = omics.create_reference_store(name='omics_demo_ref_store')
    print(response)
else:
    print("Congratulations, you have an existing reference store!")

### Step 6. Importing references
Now we will import a reference using the start_reference_import_job API call. All references in a Reference store must have a unique name. So, we're also going to apply a timestamp to the reference name to ensure that it is unique.

The code below uses the reference store we created (or retrieved) and the IAM role we created above. 

<div class="alert alert-block alert-info">
<b>NOTE:</b> Before running the cell below make sure to replace [YOUR-BUCKET] with name of your S3 bucket.</div>

In [ ]:
# set a timestamp
dt_fmt = '%Y%m%dT%H%M%S'
ts = datetime.now().strftime(dt_fmt)

ref_name = f'scrnaseq-demo-ref-{ts}'
ref_uri = 's3://[YOUR-BUCKET]/GRCm38.p6.genome.chr19.fa' #Replace [YOUR-BUCKET] with name of your S3 bucket

ref_import_job = omics.start_reference_import_job(
    referenceStoreId=get_ref_store_id(omics),
    roleArn=get_role_arn(omics_iam_name),
    sources=[{
        'sourceFile': ref_uri,
        'name': ref_name,
    }])

#The import can take up to 5 minutes to complete. We can wait for it to complete using a waiter.
print(f"waiting for job {ref_import_job['id']} to complete")
try:
    waiter = omics.get_waiter('reference_import_job_completed')
    waiter.wait(referenceStoreId=ref_import_job['referenceStoreId'], id=ref_import_job['id'])

    print(f"job {ref_import_job['id']} complete")
except botocore.exceptions.WaiterError as e:
    print(f"job {ref_import_job['id']} FAILED:")
    print(e)

------------------------------------------------------
# Sequence Store
Now we need to create a sequence store, which is similar to an S3 bucket and holds a set of objects called read sets that can be in various formats (i.e., FASTQ, BAM, CRAM). <br>
Again, for the purposes of this demo we will not read in data from the sequence store, but this shows you the process of creating a store and importing read sets if you want to utilize them in your pipeline(s).

### Step 7. Create Sequence Store

In [ ]:
# set a timestamp
dt_fmt = '%Y%m%dT%H%M%S'
ts = datetime.now().strftime(dt_fmt)

sequence_store_name = f'omics-demo-scrnaseq-store-{ts}'
response = omics.create_sequence_store(name=sequence_store_name)

In [ ]:
#Create sequence store and print name
seqstore = response
print(seqstore['name'])

### Step 8a. Import Read Sets

In [ ]:
#import demo fastq files as read sets into newly created sequence store
source1 = 's3://aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0/Sample_X_S1_L001_R1_001.fastq.gz'
source2 = 's3://aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0/Sample_X_S1_L001_R2_001.fastq.gz'

readset_import_job = omics.start_read_set_import_job(
    roleArn=get_role_arn(omics_iam_name),
    sequenceStoreId=seqstore['id'], 
        sources=[
        {
            'sourceFiles': {
                'source1': source1,
                'source2': source2
            },
            'sourceFileType': 'FASTQ',
            'subjectId': 'demo_subject',
            'sampleId': 'demo_sample',
            'generatedFrom': 'nf-core scrnaseq test data',
            'name': 'demo',
            'description': 'FASTQ for scrnaseq demo',
        },
    ]
)

#Monitor import and wait for completion using a waiter.
print(f"waiting for job {readset_import_job['id']} to complete")
try:
    waiter = omics.get_waiter('read_set_import_job_completed')
    waiter.wait(sequenceStoreId=readset_import_job['sequenceStoreId'], id=readset_import_job['id'])

    print(f"job {readset_import_job['id']} complete")
except botocore.exceptions.WaiterError as e:
    print(f"job {readset_import_job['id']} FAILED:")
    print(e)

### Step 8b. Alternatively download samples to S3 bucket

In [ ]:
#Region e.g. us-east-1
#Buckey e.g. s3://nigms-scrnaseq-bucket-demo-data 
!aws s3 cp --recursive s3://aws-genomics-static-[YOUR-REGION]/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0 s3://[YOUR-BUCKET] #Replace [YOUR-REGION] and [YOUR-BUCKET]

### Step 9. Update sample sheet with appropriate uris
Download the samplesheet file stored in your s3 bucket and replace the paths with the uri3 paths from Step 8a (if using sequence store) or Step 8b if using data copied directly to your s3 bucket.

### Step 10. Create *input.json* file
Use template text below to create *input.json* file and upload into the parameters folder. Make sure to **change [YOUR-BUCKET]** to the name of the S3 bucket you created and stored your data in.

```json
{
        "input": "s3://[YOUR-BUCKET]/samplesheet-2-0.csv",
        "protocol": "10XV2",
        "aligner": "star",
        "fasta": "s3://[YOUR-BUCKET]/GRCm38.p6.genome.chr19.fa",
        "gtf": "s3://[YOUR-BUCKET]/gencode.vM19.annotation.chr19.gtf"
}
```

## Well done! Now you're ready to create and run a private workflow!